In [1]:
import QUANTAXIS as qa
from QIFIAccount import QIFI_Account,QA_Position
from QIFIAccount.QASimAccount import QIFI_SIM_Account
import pandas as pd

提示：当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口 panel 参数将固定为 False
注意：0.25 以上版本 pandas 不支持 panel，如使用该数据结构和相关函数请注意修改


In [2]:
my=QIFI_SIM_Account(username='myacc6',password='myacc6')
my.initial()

Create new Account


In [3]:
df=pd.read_excel("E:\\leo\\投资\\2021\\股票持仓分析20210509-快照20210520.xlsx",sheet_name="持仓明细详情",header=6)
df=df[df['pos_num']>0]
code_cn_list=list(x.split(".")[0] for x in df.code[df['currency']=='CNY'].unique().tolist())
code_cn_list
code_hk_list=list(x.split(".")[0] for x in df.code[df['currency']=='HK'].unique().tolist())
code_hk_list=[i.rjust(5,'0') if len(i)==4  else i for i in code_hk_list]

In [4]:
def add_position(stock):
    code=stock['code'].split('.')
    if len(code[0])==6 and stock['currency']=='CNY':
        code=code[0]
        market_type=qa.MARKET_TYPE.STOCK_CN
        exchange_id=qa.EXCHANGE_ID.SZSE if stock['code'][0] in (0,3) else qa.EXCHANGE_ID.SSE
    elif len(code[0])==6:
        code=code[0]
        market_type='stock_cn_b'
        exchange_id=qa.EXCHANGE_ID.SZSE
    else:
        code=str(code[0]).rjust(5,'0')
        market_type='stock_hk'
        exchange_id='hk'
    pos=QA_Position(
                code=code,
                account_cookie='myacc6',
                market_type=market_type,
                exchange_id=exchange_id,
                volume_long_his=stock['pos_num'],
                volume_long_today=0,
                open_price_long=stock['cost'],
                name=stock['name'])
    my.add_position(pos)
def s(x):
    print(type(x['code']),x['code'].split('.'))

In [5]:
df.groupby('code').agg({'pos_num':'sum','cost':'mean','name':'first','currency':'first'}).reset_index().apply(lambda x:add_position(x),axis=1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
dtype: object

In [7]:
day='2021-06-03'
for code in code_cn_list:
    price=qa.QA_fetch_get_stock_day('tdx',code,day,day)
    price.apply(lambda x:my.on_price_change(code,x.close,datetime=x.date),axis=1)
for code in code_hk_list:
    price=qa.QA_fetch_get_hkstock_day('tdx',code,day,day) if len(code)==5 else qa.QA_fetch_get_stock_day('tdx',code,day,day)
    price.apply(lambda x:my.on_price_change(code,x.close*0.8244,datetime=x.date),axis=1)

QUANTAXIS>> Selecting the Best Server IP of TDX


USING DEFAULT STOCK IP
USING DEFAULT FUTURE IP


QUANTAXIS>> === The BEST SERVER ===
 stock_ip 123.125.108.23 future_ip 106.14.95.149


In [14]:
my.ask_deposit(121212)

In [9]:
my.settle()

settle


In [8]:
my.balance

1411443.25893413

In [12]:
df1[~(df1['exchange_id']=='hk')]['balance'].sum()

1695961.5682

In [13]:
df1['position_profit'].sum()

411443.2589341301

In [11]:
df=pd.DataFrame(my.position_msg).T
df1=df[['code','name','exchange_id','volume_long_his','pos_long_his','open_price_long','last_price','position_profit','position_cost_long']].assign(balance=df['last_price']*df['pos_long_his'])
df1

,code,name,exchange_id,volume_long_his,pos_long_his,open_price_long,last_price,position_profit,position_cost_long,balance
stock_cn.000333,000333,美的集团,sse,374,374,51.69,77.93,9813.76,19332.1,29145.8
stock_cn.000961,000961,中南建设,sse,13500,13500,7.256,6.79,-6291,97956,91665
stock_cn.002027,002027,分众传媒,sse,12700,12700,5.986,9.76,47929.8,76022.2,123952
stock_cn.002353,002353,杰瑞股份,sse,300,300,32.265,40.67,2521.5,9679.5,12201
stock_cn.00700,00700,腾讯控股,hk,400,400,474.321,507.418,13239.1,189728,202967
stock_cn.01088,01088,中国神华,hk,12000,12000,14.96,14.4765,-5802.42,179520,173718
stock_cn.01516,01516,融创服务,hk,32,32,17.16,19.6619,80.0621,549.12,629.182
stock_cn.159920,159920,恒生ETF,sse,44700,44700,1.525,1.495,-1341,68167.5,66826.5
stock_cn.160322,160322,港股精选,sse,2800,2800,1.796,1.641,-434,5028.8,4594.8
stock_cn.160631,160631,银行指基,sse,2447,2447,1,1.067,163.949,2447,2610.95


In [97]:
df.T

,stock_cn.000333,stock_cn.000961,stock_cn.002027,stock_cn.002353,stock_cn.00700,stock_cn.01088,stock_cn.01516,stock_cn.159920,stock_cn.160322,stock_cn.160631,...,stock_cn.510900,stock_cn.512800,stock_cn.600036,stock_cn.600516,stock_cn.600985,stock_cn.601166,stock_cn.601225,stock_cn.601318,stock_cn.601668,stock_cn.601939
code,000333,000961,002027,002353,00700,01088,01516,159920,160322,160631,...,510900,512800,600036,600516,600985,601166,601225,601318,601668,601939
instrument_id,000333,000961,002027,002353,00700,01088,01516,159920,160322,160631,...,510900,512800,600036,600516,600985,601166,601225,601318,601668,601939
user_id,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,...,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5
portfolio_cookie,portfolio,portfolio,portfolio,portfolio,portfolio,portfolio,portfolio,portfolio,portfolio,portfolio,...,portfolio,portfolio,portfolio,portfolio,portfolio,portfolio,portfolio,portfolio,portfolio,portfolio
username,quantaxis,quantaxis,quantaxis,quantaxis,quantaxis,quantaxis,quantaxis,quantaxis,quantaxis,quantaxis,...,quantaxis,quantaxis,quantaxis,quantaxis,quantaxis,quantaxis,quantaxis,quantaxis,quantaxis,quantaxis
position_id,8821b070-7b24-4952-9654-7dadb9898442,c696f1ca-5e96-4ba6-be54-f4d1101e34f8,08714c96-6a77-4539-8a65-0cde00686232,e37682b3-b27c-42f2-a1e9-04ffb6b7aac2,5bf9d09d-3838-4881-a399-21df8a86aea9,944b8ff7-c6b9-4e3f-99e0-78c23261e717,f26ae12d-43ee-43c6-8144-010c8dae1e9a,e9ddad5f-980a-46ca-8276-b2d9b96fb6f8,9800273f-e646-4195-b79c-8a14271238bd,975faf1c-25d2-482d-b00a-cdc55fc6bc0e,...,8e4be656-782a-4f0b-9f44-eb8e0fb6d040,0fc71f72-a97d-42dd-b9cc-4d49cdbc64fe,04a1f99c-b390-40f7-b619-f3ffb3fad5ff,1a4d2ea0-558a-4499-b977-551b57d5fea7,9d74789f-5234-4cd7-9224-ac983ee37ee6,86117bb3-c1f9-4131-b060-996eef6f9816,4519e467-a9b0-4d94-b098-4944467a492b,591469e9-a181-4d12-8380-9161aa306a2c,8f5f8da3-23df-407c-9d1d-60dd64537bdd,90e83cc1-6b2f-4125-b03c-f523bdb144a0
account_cookie,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,...,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5,myacc5
frozen,{},{},{},{},{},{},{},{},{},{},...,{},{},{},{},{},{},{},{},{},{}
name,美的集团,中南建设,分众传媒,杰瑞股份,腾讯控股,中国神华,融创服务,恒生ETF,港股精选,银行指基,...,H股ETF,银行ETF,招商银行,方大炭素,淮北矿业,兴业银行,陕西煤业,中国平安,中国建筑,建设银行
spms_id,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [80]:
df.groupby('code').agg({'pos_num':'sum','cost':'mean','name':'first','currency':'first'}).reset_index().apply(lambda x:add_position(x),axis=1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
dtype: object

In [103]:
my.balance

1411443.25893413

In [ ]:
def get_pos(stock):
    return QA_Position(
        code=stock['code'],
        account_cookie='myacc5',
        market_type=qa.MARKET_TYPE.STOCK_CN,
        exchange_id=qa.EXCHANGE_ID.SZSE if stock['code'][0] in (0,3) else qa.EXCHANGE_ID.SSE,
        volume_long_his=stock['pos'],
        volume_long_today=0,
        open_price_long=stock['price'],
        name=stock['name']
    )

In [ ]:
for pos in cn_stock.apply(lambda x:get_pos(x),axis=1):
    my.add_position(pos)